In [1]:

# !Z conda activate tf_cpu
# This code use 
#   BertForSequenceClassification
#   bert-base-uncased (where cat & CAT are the same)


# conda create --name tf_cpu python=3.9
# conda activate tf_cpu
# pip install tensorflow (=> 2.17.0)y
# conda install tf-keras (=> 2.15)
# conda install transformers (=> 4.44)

# conda install transformers (=> 4.44 + keras 3.5 qui n'est pas supporté)
#       Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
#       Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.
# pip uninstall keras

## pip install transformers==4.29.0
## pip install tokenizers==0.13.3

# prelude
import pandas as pd
# import matplotlib.pyplot as plt
import tensorflow as tf
# import numpy as np

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard


from transformers import TFBertForSequenceClassification, BertTokenizer # 'TF' for TensorFlow models
# , trainer, TrainingArguments           
from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

from pathlib import Path
k_Current_dir = Path.cwd()
k_AssetsDir = "assets"
k_sms_max_len = 100
k_random_state  = 42
k_test_size     = 0.3


# Regarding the warning : TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html from .autonotebook import tqdm as notebook_tqdm
# Solution ? 
# ! NOT TESTED YET - Does it apply to Jupyter "in" VSCode ?
# https://saturncloud.io/blog/importerror-iprogress-not-found-please-update-jupyter-and-ipywidgets-although-it-is-installed/
# conda install -c conda-forge ipywidgets
# jupyter nbextension enable --py widgetsnbextension

c:\Users\phili\anaconda3\envs\tf_cpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
DLL load failed while importing _errors: La procédure spécifiée est introuvable.

In [6]:
# -----------------------------------------------------------------------------
# drop empty cols and duplicates, rename cols...
def cleaner(df):
    df.drop(columns="Unnamed: 2", inplace=True)
    df.drop(columns="Unnamed: 3", inplace=True)
    df.drop(columns="Unnamed: 4", inplace=True)

    df.drop_duplicates(inplace=True)

    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace("/", "_")

    df.rename(columns={"v1": "label"}, inplace=True)
    df.rename(columns={"v2": "sms"}, inplace=True)

    df["label"] = df["label"].map({"ham": 0, "spam": 1})

    return df

In [7]:
# -----------------------------------------------------------------------------
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = Path(f"{k_Current_dir/k_AssetsDir/fig_id}.{fig_extension}")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    return

In [ ]:
df = pd.read_csv(k_Current_dir / k_AssetsDir / "spam.csv", encoding="cp1252")
df = cleaner(df)

X_train, X_test, y_train, y_test = train_test_split(df['sms'], df['label'], test_size=k_test_size, random_state=k_random_state, stratify=df['label'])

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # modèle TensorFlow
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# On ne peut pas utiliser la classe trainer de Pytorch
# Il faut utiliser les outils TensorFlow

early_stopping = EarlyStopping(
    monitor='val_loss',           # can be 'val_accuracy' if needed 
    patience=3,          
    restore_best_weights=True  
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2,       # reduction factor of learning rate
    patience=2,       
    min_lr=1e-7       # minimal value for learning rate
)

path = Path(f"{k_Current_dir/k_AssetsDir/'bert_base_uncased_best_model.h5'}")
checkpoint = ModelCheckpoint(
    path,                       # model's path
    monitor='val_loss', 
    save_best_only=True, 
    mode='min'
)

tensorboard = TensorBoard(log_dir='logs', histogram_freq=1)
print(f"\n\n--------------------------------------------------")
print(f"Once the model runs, open a terminal, make sure you are in the directory of the project and type in : ")
print(f"tensorboard --logdir=logs")
print(f"Then visit the URL")

model.compile(
    optimizer=Adam(learning_rate=3e-5), 
    loss='binary_crossentropy', 
    metrics=[tf.keras.metrics.Recall(name="recall"), tf.keras.metrics.Precision(name="precision"), "accuracy"],       # name=... avoid recall_1 for example
)

# Tokenisation des textes pour l'entrée du modèle
train_encodings = tokenizer(
    X_train.tolist(),
    truncation=True,           # tronque si le texte est trop long pour BERT
    padding=True,              # ajoute du padding pour aligner les longueurs des séquences
    max_length=128,            # longueur max des séquences
    return_tensors='tf'        # retourne des tenseurs TensorFlow
)

test_encodings = tokenizer(
    X_test.tolist(),
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors='tf'
)

# Conversion des labels en tenseurs TensorFlow
y_train_tf = tf.convert_to_tensor(y_train.tolist())
y_test_tf = tf.convert_to_tensor(y_test.tolist())

# Entraînement du modèle
history = model.fit(
    x={"input_ids": train_encodings['input_ids'], "attention_mask": train_encodings['attention_mask']},   # entrée des tokens et des masques
    y=y_train_tf,                          # les labels
    validation_data=(
        {"input_ids": test_encodings['input_ids'], "attention_mask": test_encodings['attention_mask']},  # entrée de validation
        y_test_tf                           # labels de validation
    ),
    batch_size=32,
    epochs=50,
    callbacks=[early_stopping, reduce_lr, checkpoint, tensorboard]
)


In [ ]:
path = Path(f"{k_Current_dir/k_AssetsDir/'bert_base_uncased_arch.png'}")
tf.keras.utils.plot_model(model, path, show_shapes=True)

In [ ]:
plt.plot(history.history["loss"], color="b", label="Train Loss")
plt.plot(history.history["val_loss"], color="r", label="Val Loss")
plt.ylabel("Values")
plt.xlabel("Epochs")
plt.title("Bert Base Uncased : Loss")
plt.legend()
plt.ylim(0,1)
save_fig("bert_base_uncased_loss", "png")
plt.show()

display([[round(f, 6) for f in history.history['loss'][-10:]]])
display([[round(f, 6) for f in history.history['val_loss'][-10:]]])


In [ ]:
plt.plot(history.history["accuracy"], color="b", label="Train Accuracy")
plt.plot(history.history["val_accuracy"], color="r", label="Val Accuracy")
plt.ylabel("Values")
plt.xlabel("Epochs")
plt.title("Bert Base Uncased : Accuracy")
plt.legend()
plt.ylim(0,1)
save_fig("bert_base_uncased_accuracy", "png")
plt.show()

display([[round(f, 6) for f in history.history['accuracy'][-10:]]])
display([[round(f, 6) for f in history.history['val_accuracy'][-10:]]])


In [ ]:
plt.plot(history.history["precision"], color="b", label="Train Precision")
plt.plot(history.history["val_precision"], color="r", label="Val Precision")
plt.ylabel("Values")
plt.xlabel("Epochs")
plt.title("Bert Base Uncased : Precision")
plt.legend()
plt.ylim(0,1)
save_fig("bert_base_uncased_precision", "png")
plt.show()

display([[round(f, 6) for f in history.history['precision'][-10:]]])
display([[round(f, 6) for f in history.history['val_precision'][-10:]]])

In [ ]:
plt.plot(history.history["recall"], color="b", label="Train Recall")
plt.plot(history.history["val_recall"], color="r", label="Val Recall")
plt.ylabel("Values")
plt.xlabel("Epochs")
plt.title("Bert Base Uncased : Recall")
plt.legend()
plt.ylim(0,1)
save_fig("bert_base_uncased_recall", "png")
plt.show()

display([[round(f, 6) for f in history.history['recall'][-10:]]])
display([[round(f, 6) for f in history.history['val_recall'][-10:]]])

In [10]:
def f1_calculus(name, rec, prec):
    df_tmp=pd.DataFrame()
    df_tmp[name] = 2*np.array(rec)*np.array(prec)/(np.array(rec)+np.array(prec)+tf.keras.backend.epsilon()) # epsilon avoid runtimeWarning: divide by zero encountered in divide...
    return df_tmp

In [ ]:
df_tmp = f1_calculus("f1", history.history["recall"], history.history["precision"])
df_val_tmp = f1_calculus("val_f1", history.history["val_recall"], history.history["val_precision"])

plt.plot(df_tmp["f1"], color="b", label="Train F1")
plt.plot(df_val_tmp["val_f1"], color="r", label="Val F1")
plt.ylabel("Values")
plt.xlabel("Epochs")
plt.title("Bert Base Uncased : F1")
plt.legend()
plt.ylim(0,1)
save_fig("bert_base_uncased_f1", "png")
plt.show()

display(df_tmp.tail(10))
display(df_val_tmp.tail(10))